In [ ]:
import ee
ee.Initialize()

In [ ]:
ee_path = 'users/balakumaran247/swSentinel/state'

In [ ]:
s1_image = {
    'TN': ee.Image("users/balakumaran247/swSentinel/state/TN_s1_image"),
    'KA': ee.Image("users/balakumaran247/swSentinel/state/KA_s1_image"),
}
s2_image = {
    'TN': ee.Image("users/balakumaran247/swSentinel/state/TN_s2_image"),
    'KA': ee.Image("users/balakumaran247/swSentinel/state/KA_s2_image"),
}
water_samples = {
    'TN': ee.FeatureCollection("users/balakumaran247/swSentinel/state/TN_water_samples"),
    'KA': ee.FeatureCollection("users/balakumaran247/swSentinel/state/KA_water_samples"),
}
non_water_samples = {
    'TN': ee.FeatureCollection("users/balakumaran247/swSentinel/state/TN_non_water_samples"),
    'KA': ee.FeatureCollection("users/balakumaran247/swSentinel/state/KA_non_water_samples"),
}

In [ ]:
def get_values(image, points):
    return image.sampleRegions(
        collection=points,
        properties=['wClass', 'State'],
        scale=image.projection().nominalScale(),
        geometries=True
    )

In [ ]:
def export_drive(samples, state_abb, class_name, image_name):
    task = ee.batch.Export.table.toDrive(
        collection=samples,
        description=f'{image_name}_{state_abb}_{class_name}',
        fileFormat='CSV')
    # task.start()

In [ ]:
def main(state_abb):
    values = [(get_values(s1_image[state_abb], water_samples[state_abb]), 'water', 'S1'),
    (get_values(s2_image[state_abb], water_samples[state_abb]), 'water', 'S2'),
    (get_values(s1_image[state_abb], non_water_samples[state_abb]), 'nonWater', 'S1'),
    (get_values(s2_image[state_abb], non_water_samples[state_abb]), 'nonWater', 'S2')]
    
    for fc, cname, simage in values:
        export_drive(fc, state_abb, cname, simage)

In [ ]:
main('TN')